In [13]:
using RigidBodyDynamics
using LinearAlgebra
using StaticArrays
using MeshCat
using MeshCatMechanisms
using MechanismGeometries

include("/home/hkolano/onr-dynamics-julia/simulate_with_ext_forces.jl")
println("Libraries imported.")

Libraries imported.


In [24]:
# ------------------------------------------------------------------------
#                            MODEL DEFINITION
# ------------------------------------------------------------------------
# Define paths to the URDF files
src_dir = dirname(pathof(RigidBodyDynamics))
urdf_file_alpha = joinpath(src_dir, "..", "..", "..", "..", "..", "onr-dynamics-julia", "URDFs", "alphaArmFixedJaws.urdf")
urdf_file_water = joinpath(src_dir, "..", "..", "..", "..", "..", "onr-dynamics-julia", "URDFs", "alphaArmWater.urdf")
mechanism_alpha = parse_urdf(urdf_file_alpha)
mechanism_water = parse_urdf(urdf_file_water; gravity = [0.0, 0.0, 0.0])

# Create visuals of the URDFs
visuals = URDFVisuals(urdf_file_alpha)
water_vis = URDFVisuals(urdf_file_water)
mvis_alpha = MechanismVisualizer(mechanism_alpha, URDFVisuals(urdf_file_alpha))
mvis_water = MechanismVisualizer(mechanism_water, URDFVisuals(urdf_file_water))

# Name the joints of the mechanisms
base_joint, shoulder_joint, elbow_joint, wrist_joint = joints(mechanism_alpha)
base_j_water, shoulder_j_water, elbow_j_water, wrist_j_water = joints(mechanism_water)

println("URDFs parsed. Visualizers ready. \n")

URDFs parsed. Visualizers ready. 



┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8706
└ @ MeshCat /home/hkolano/.julia/packages/MeshCat/nXUuG/src/visualizer.jl:69
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8707
└ @ MeshCat /home/hkolano/.julia/packages/MeshCat/nXUuG/src/visualizer.jl:69


In [25]:
# ------------------------------------------------------------------------
#                          INITIAL CONDITIONS
# ------------------------------------------------------------------------
function impose_alpha_state_on_water(state_alpha, state_water)
    """
    Take the current state (configuration and velocity) of the alpha arm and impose it on the water mechanism state.
    """
    set_configuration!(state_water, base_j_water, configuration(state_alpha, base_joint))
    set_configuration!(state_water, shoulder_j_water, configuration(state_alpha, shoulder_joint))
    set_configuration!(state_water, elbow_j_water, configuration(state_alpha, elbow_joint))
    set_configuration!(state_water, wrist_j_water, configuration(state_alpha, wrist_joint))
    set_velocity!(state_water, base_j_water, velocity(state_alpha, base_joint))
    set_velocity!(state_water, shoulder_j_water, velocity(state_alpha, shoulder_joint))
    set_velocity!(state_water, elbow_j_water, velocity(state_alpha, elbow_joint))
    set_velocity!(state_water, wrist_j_water, velocity(state_alpha, wrist_joint))
end;

function impose_alpha_desacc_on_water(des_acc, des_acc_water)
    des_acc_water[base_j_water][1] = des_acc[base_joint][1]
    des_acc_water[shoulder_j_water][1] = des_acc[shoulder_joint][1]
    des_acc_water[elbow_j_water][1] = des_acc[elbow_joint][1]
    des_acc_water[wrist_j_water][1] = des_acc[wrist_joint][1]
end;

# Initialize state of alpha arm
state_alpha = MechanismState(mechanism_alpha)
zero!(state_alpha)
# set_configuration!(state, base_joint, 0.0)
# set_configuration!(state, shoulder_joint, -.27079)
# set_configuration!(state, elbow_joint, 3.1415/2+1.3)
# set_configuration!(state, wrist_joint, 0.5)

# Initialize state of water arm
state_water = MechanismState(mechanism_water)
# Make water state = arm state
impose_alpha_state_on_water(state_alpha, state_water)
# state_water_no_vel = deepcopy(state_water)

des_acc = similar(velocity(state_alpha))
des_acc[base_joint][1] = 0.5
des_acc[shoulder_joint][1] = 0.
des_acc[elbow_joint][1] = 0.
des_acc[wrist_joint][1] = 0.

des_acc_water = similar(velocity(state_water))
impose_alpha_desacc_on_water(des_acc, des_acc_water)

0.0

In [26]:
# ------------------------------------------------------------------------
#                              CONTROLLER
# ------------------------------------------------------------------------

function simple_control!(torques::AbstractVector, t, state_alpha::MechanismState)
    # Calculate inverse dynamics of alpha arm
    tau = inverse_dynamics(state_alpha, des_acc)
    # Find dynamic terms for water model
    impose_alpha_state_on_water(state_alpha, state_water)
    tau_water = inverse_dynamics(state_water, des_acc_water)
    # tau_water_bias = dynamics_bias(state_water)

    # set_configuration!(state_water_no_vel) = configuration(state_water)
    # zero_velocity!(state_water_no_vel)
    # tau_water_bias_grav_only = dynamics_bias(state_water_no_vel)

    # # M_a and C_a
    # tau_water_wo_grav = tau_water - tau_water_bias_grav_only  
 

    # Set torques accordingly (controller)
    torques[velocity_range(state_alpha, base_joint)] .= tau[base_joint][1]
    torques[velocity_range(state_alpha, shoulder_joint)] .= tau[shoulder_joint][1]
    torques[velocity_range(state_alpha, elbow_joint)] .= tau[elbow_joint][1]
    torques[velocity_range(state_alpha, wrist_joint)] .= tau[wrist_joint][1]
end;

In [27]:
# ------------------------------------------------------------------------
#                              SIMULATION
# ------------------------------------------------------------------------
final_time = 1.
ts, qs, vs = simulate(state_alpha, final_time, simple_control!; Δt = 1e-2)

println("Simulation finished.")

Checking Water Taus
[0.0036816, -1.936e-5, 6.31672e-6, -1.08157e-5]
[0.0036816, -1.936e-5, 6.31672e-6, -1.08157e-5]
Checking Water Taus
[0.0036816, -1.93484e-5, 6.31449e-6, -1.08157e-5]
[0.0036816, -1.93484e-5, 6.31449e-6, -1.08157e-5]
Checking Water Taus
[0.0036816, -1.93484e-5, 6.31449e-6, -1.08157e-5]
[0.0036816, -1.93484e-5, 6.31449e-6, -1.08157e-5]
Checking Water Taus
[0.0036816, -1.93137e-5, 6.3078e-6, -1.08157e-5]
[0.0036816, -1.93137e-5, 6.3078e-6, -1.08157e-5]
Checking Water Taus
[0.0036816, -1.93137e-5, 6.3078e-6, -1.08157e-5]
[0.0036816, -1.93137e-5, 6.3078e-6, -1.08157e-5]
Checking Water Taus
[0.0036816, -1.92558e-5, 6.29665e-6, -1.08157e-5]
[0.0036816, -1.92558e-5, 6.29665e-6, -1.08157e-5]
Checking Water Taus
[0.0036816, -1.92558e-5, 6.29665e-6, -1.08157e-5]
[0.0036816, -1.92558e-5, 6.29665e-6, -1.08157e-5]
Checking Water Taus
[0.0036816, -1.91747e-5, 6.28104e-6, -1.08157e-5]
[0.0036816, -1.91747e-5, 6.28104e-6, -1.08157e-5]
Checking Water Taus
[0.0036816, -1.91747e-5, 6.2

In [8]:
# ------------------------------------------------------------------------
#                             VISUALIZATION
# ------------------------------------------------------------------------
# vis = Visualizer()
# open(vis)

# delete!(vis)
# mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf_file))
# set_configuration!(mvis, [0.0, 0.0])
# open(mvis)
MeshCatMechanisms.animate(mvis_alpha, ts, qs; realtimerate = 1.)

# animation = Animation(mvis, ts, qs)
# setanimation!(mvis, animation)

println("\n done.")


 done.
